# German Credit Dataset

# CGAN Training for synthesize datasets


CGAN: A conditional generative adversarial network (CGAN) is a type of GAN that also takes advantage of labels during the training process. Generator — Given a label and random array as input, this network generates data with the same structure as the training data observations corresponding to the same label.

Then, we save the CGAN models for the data generation pipeline.

## Imports

In [2]:
import os
import pandas as pd
import numpy as np
from ydata_synthetic.synthesizers.regular import CGAN
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# from src.pipeline.data_generation.data_generator import GANDataGenerator
from src.pipeline.datasets.training_datasets import GermanCreditDataset
from src.pipeline.preprocessing.label_preprocessor import LabelProcessor
from src.pipeline.preprocessing.paths import GERMAN_CREDIT_LABEL_ENCODER_PATH
from src.pipeline.model.paths import GERMAN_CREDIT_GEN_CGAN_MODEL_PATH


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Preprocessed Data

In [4]:
# init GANDataGenerator
print('German Credit dataset\n')

dataset = GermanCreditDataset()
processor = LabelProcessor(dataset, GERMAN_CREDIT_LABEL_ENCODER_PATH)
df = processor.preprocessed_data(dataset.raw_df)
label_col = dataset.label_column_name

df.head()
df.shape

print(f'Label columns name is: {label_col}. With {df[label_col].nunique()} unique values.'
      f'({df[label_col].unique()})')



German Credit dataset

loading dataset


/home/user/code/data_drift_detctor/src/pipeline/config.py:21: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  cfg = ryaml.load(f)


,existingchecking,duration,credithistory,purpose,creditamount,savings,employmentsince,installmentrate,statussex,otherdebtors,...,property,age,otherinstallmentplans,housing,existingcredits,job,peopleliable,telephone,foreignworker,classification
0,0,6,4,4,1169,4,4,4,2,0,...,0,67,2,1,2,2,1,1,0,0
1,1,48,2,4,5951,0,2,2,1,0,...,0,22,2,1,1,2,1,0,0,1
2,3,12,4,7,2096,0,3,2,2,0,...,0,49,2,1,1,1,2,0,0,0
3,0,42,2,3,7882,0,3,2,2,2,...,1,45,2,2,1,2,2,0,0,0
4,0,24,3,0,4870,0,2,3,2,0,...,3,53,2,2,2,2,2,0,0,1


(1000, 21)

Label columns name is: classification. With 2 unique values.([0 1])


# EDA and Preprocessing

In [5]:
df.shape

(1000, 21)

In [6]:
train_sample = df

# Init the GAN

In [7]:

to_save = False

#Define the Conditional GAN and training parameters

noise_dim = 32
dim = 128
batch_size = 128
beta_1 = 0.5
beta_2 = 0.9

log_step = 100
epochs = 300 + 1
learning_rate = 5e-4


gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           betas=(beta_1, beta_2),
                           noise_dim=noise_dim,
                           n_cols=train_sample.shape[1] - 1,  # Don't count the label columns here
                           layers_dim=dim)

train_args = TrainParameters(epochs=epochs,
                             cache_prefix='',
                             sample_interval=log_step,
                             label_dim=-1,
                             labels=[0,1])

num_classes = df[label_col].nunique()

#Init the Conditional GAN providing the index of the label column as one of the arguments
synthesizer = CGAN(model_parameters=gan_args, num_classes=num_classes)




2021-12-25 21:38:21.011413: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-25 21:38:21.011978: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-25 21:38:21.057770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-25 21:38:21.058183: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro RTX 5000 with Max-Q Design computeCapability: 7.5
coreClock: 1.35GHz coreCount: 48 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 357.69GiB/s
2021-12-25 21:38:21.058218: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-12-25 21:38:21.059305: I tensorflow/stream_executor/p

# Training


In [9]:
#----------------------------
#    GAN Training
#----------------------------

#Training the Conditional GAN
synthesizer.train(data=train_sample, label_col=label_col, train_arguments=train_args,
                 num_cols=dataset.numeric_feature_names, cat_cols=dataset.categorical_feature_names )



Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-12-25 21:38:37.601935: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-25 21:38:37.603123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-25 21:38:37.603761: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro RTX 5000 with Max-Q Design computeCapability: 7.5
coreClock: 1.35GHz coreCount: 48 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 357.69GiB/s
2021-12-25 21:38:37.603794: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic l

0 [D loss: 0.688936, acc.: 83.59%] [G loss: 0.701787]



  1%|          | 2/301 [00:02<05:43,  1.15s/it]

1 [D loss: 0.661413, acc.: 87.89%] [G loss: 0.755089]


...

 99%|█████████▉| 299/301 [02:39<00:01,  1.89it/s]

298 [D loss: 0.000009, acc.: 100.00%] [G loss: 116.679039]


100%|█████████▉| 300/301 [02:40<00:00,  1.91it/s]

299 [D loss: 0.000001, acc.: 100.00%] [G loss: 114.558273]
300 [D loss: 0.000001, acc.: 100.00%] [G loss: 129.603546]



100%|██████████| 301/301 [02:41<00:00,  1.86it/s]


# Save the model

In [11]:
lock = True
if lock:
    synthesizer.save(GERMAN_CREDIT_GEN_CGAN_MODEL_PATH)

# Synthesize samples based on the trained CGAN:

In [73]:
synthesizer.sample(condition=np.array([1]), n_samples=2).shape

Synthetic data generation: 100%|██████████| 1/1 [00:00<00:00, 18.97it/s]


(128, 52)

In [3]:
synthesizer = CGAN.load(GERMAN_CREDIT_GEN_CGAN_MODEL_PATH)

NameError: name 'CGAN' is not defined

# Generate Samples

In [13]:
synthesizer.generator.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(128, 1)]           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (128, 1, 1)          2           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(128, 32)]          0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (128, 1)             0           embedding[0][0]                  
______________________________________________________________________________________________

In [12]:
synthesizer.discriminator.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(128, 1)]           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(128, 115)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (128, 1, 1)          2           input_4[0][0]                    
__________________________________________________________________________________________________
flatten_2 (Flatten)             (128, 115)           0           input_3[0][0]                    
____________________________________________________________________________________________

# Load Trained Model

In [7]:
synthesizer = CGAN.load(GERMAN_CREDIT_GEN_CGAN_MODEL_PATH)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-12-25 11:19:07.241876: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-25 11:19:07.242590: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-25 11:19:07.267683: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2021-12-25 11:19:07.267708: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: data-science-l-danielle
2021-12-25 11:19:07.267714: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: data-science-l-danielle
2021-12-25 11:19:07.267783: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.86.0
2021-12-25 11:19:07.267802: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.86.0
2021-12-25 11:19:07.267807: I tensorflow/stream_execut

In [29]:
np.ceil(200/128)

2.0

In [48]:
generated_df_class_true = synthesizer.sample(condition=np.array([1]), n_samples=1)#batch_size*100)
generated_df_class_true

Synthetic data generation: 100%|██████████| 1/1 [00:00<00:00, 14.20it/s]


,duration,creditamount,installmentrate,residencesince,age,existingcredits,peopleliable,existingchecking_A11,existingchecking_A12,existingchecking_A13,...,housing_A153,job_A171,job_A172,job_A173,job_A174,telephone_A191,telephone_A192,foreignworker_A201,foreignworker_A202,NaN
0,8,1055,1,1,22,1,1,1,1,0,...,0,0,1,0,1,1,0,0,0,1.0
1,8,1107,1,1,22,1,1,1,0,0,...,0,1,1,0,1,0,0,0,1,1.0
2,8,1082,1,1,22,1,1,1,1,1,...,1,0,0,1,0,1,1,0,1,1.0
3,8,1091,1,1,22,1,1,1,1,0,...,1,0,1,0,1,0,1,0,1,1.0
4,8,1100,1,1,22,1,1,0,1,0,...,1,1,1,0,0,1,1,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,8,1072,1,1,22,1,1,0,0,1,...,0,1,0,1,1,1,0,1,0,1.0
124,8,1110,1,1,22,1,1,1,1,0,...,0,0,1,1,1,1,1,1,0,1.0
125,8,1119,1,1,22,1,1,1,0,1,...,0,0,1,1,1,1,0,0,0,1.0
126,8,1073,1,1,22,1,1,1,1,0,...,0,1,0,0,1,1,1,1,0,1.0


In [49]:
generated_df_class_false = synthesizer.sample(condition=np.array([0]), n_samples=1) #n_samples=batch_size*10)
generated_df_class_false

Synthetic data generation: 100%|██████████| 1/1 [00:00<00:00, 14.05it/s]


,duration,creditamount,installmentrate,residencesince,age,existingcredits,peopleliable,existingchecking_A11,existingchecking_A12,existingchecking_A13,...,housing_A153,job_A171,job_A172,job_A173,job_A174,telephone_A191,telephone_A192,foreignworker_A201,foreignworker_A202,NaN
0,8,1087,1,1,22,1,1,0,1,1,...,1,0,1,1,0,0,1,1,0,0.0
1,8,1085,1,1,22,1,1,0,1,1,...,1,0,1,1,1,1,1,0,0,0.0
2,8,1083,1,1,22,1,1,1,0,1,...,0,0,1,1,0,0,0,1,0,0.0
3,8,1085,1,1,22,1,1,0,1,1,...,0,0,1,0,1,0,0,1,0,0.0
4,8,1082,1,1,22,1,1,0,1,1,...,0,1,0,1,0,1,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,8,1086,1,1,22,1,1,1,1,0,...,0,1,0,1,1,0,0,0,0,0.0
124,8,1084,1,1,22,1,1,1,0,1,...,1,0,0,0,0,0,1,1,1,0.0
125,8,1081,1,1,22,1,1,1,0,0,...,0,0,1,1,1,0,0,0,0,0.0
126,8,1084,1,1,22,1,1,1,1,0,...,0,1,1,1,0,1,0,0,0,0.0


In [51]:
pd.concat([generated_df_class_false, generated_df_class_true]).sample(10)

,duration,creditamount,installmentrate,residencesince,age,existingcredits,peopleliable,existingchecking_A11,existingchecking_A12,existingchecking_A13,...,housing_A153,job_A171,job_A172,job_A173,job_A174,telephone_A191,telephone_A192,foreignworker_A201,foreignworker_A202,NaN
97,8,1038,1,1,22,1,1,1,0,0,...,0,0,1,0,0,0,1,1,1,1.0
94,8,1082,1,1,22,1,1,1,1,1,...,1,1,1,1,1,0,1,0,1,0.0
44,8,1086,1,1,22,1,1,1,1,0,...,0,0,0,0,0,0,1,0,0,0.0
42,8,1126,1,1,22,1,1,1,0,0,...,0,1,1,0,0,0,1,0,0,1.0
3,8,1085,1,1,22,1,1,0,1,1,...,0,0,1,0,1,0,0,1,0,0.0
124,8,1084,1,1,22,1,1,1,0,1,...,1,0,0,0,0,0,1,1,1,0.0
2,8,1083,1,1,22,1,1,1,0,1,...,0,0,1,1,0,0,0,1,0,0.0
23,8,1078,1,1,22,1,1,1,1,1,...,1,1,0,1,0,0,1,1,0,0.0
45,8,1119,1,1,22,1,1,1,1,0,...,0,1,0,1,1,0,1,0,0,1.0
84,8,1100,1,1,22,1,1,1,0,1,...,0,1,0,1,1,1,1,1,0,1.0


In [41]:
generated_df_class_true.describe()

,duration,creditamount,installmentrate,residencesince,age,existingcredits,peopleliable,existingchecking_A11,existingchecking_A12,existingchecking_A13,...,housing_A153,job_A171,job_A172,job_A173,job_A174,telephone_A191,telephone_A192,foreignworker_A201,foreignworker_A202,NaN
count,128.0,128.000000,128.0,128.0,128.0,128.0,128.0,128.000000,128.000000,128.000000,...,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.0
mean,8.0,1101.078125,1.0,1.0,22.0,1.0,1.0,0.539062,0.445312,0.562500,...,0.531250,0.468750,0.445312,0.476562,0.570312,0.406250,0.484375,0.515625,0.484375,1.0
std,0.0,27.126885,0.0,0.0,0.0,0.0,0.0,0.500430,0.498953,0.498028,...,0.500983,0.500983,0.498953,0.501413,0.496977,0.493062,0.501719,0.501719,0.501719,0.0
min,8.0,1043.000000,1.0,1.0,22.0,1.0,1.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,8.0,1081.750000,1.0,1.0,22.0,1.0,1.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,8.0,1102.000000,1.0,1.0,22.0,1.0,1.0,1.000000,0.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0
75%,8.0,1118.000000,1.0,1.0,22.0,1.0,1.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
max,8.0,1174.000000,1.0,1.0,22.0,1.0,1.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [22]:
real_df_class_true = df[df[label_col]==1]#.sample(128)
real_df_class_true.describe()

,duration,creditamount,installmentrate,residencesince,age,existingcredits,peopleliable,classification,existingchecking_A11,existingchecking_A12,...,housing_A152,housing_A153,job_A171,job_A172,job_A173,job_A174,telephone_A191,telephone_A192,foreignworker_A201,foreignworker_A202
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.0,300.000000,300.000000,...,300.000000,300.000000,300.000000,300.000000,300.000000,300.00000,300.000000,300.000000,300.000000,300.000000
mean,24.860000,3938.126667,3.096667,2.850000,33.963333,1.366667,1.153333,1.0,0.450000,0.350000,...,0.620000,0.146667,0.023333,0.186667,0.620000,0.17000,0.623333,0.376667,0.986667,0.013333
std,13.282639,3535.818955,1.088395,1.094605,11.222379,0.559702,0.360911,0.0,0.498325,0.477767,...,0.486197,0.354364,0.151212,0.390295,0.486197,0.37626,0.485360,0.485360,0.114889,0.114889
min,6.000000,433.000000,1.000000,1.000000,19.000000,1.000000,1.000000,1.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,12.000000,1352.500000,2.000000,2.000000,25.000000,1.000000,1.000000,1.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000
50%,24.000000,2574.500000,4.000000,3.000000,31.000000,1.000000,1.000000,1.0,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,1.000000,0.000000,1.000000,0.000000
75%,36.000000,5141.500000,4.000000,4.000000,40.000000,2.000000,1.000000,1.0,1.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,1.000000,1.000000,1.000000,0.000000
max,72.000000,18424.000000,4.000000,4.000000,74.000000,4.000000,2.000000,1.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [18]:
real_df_class_false = df[df[label_col]==0].sample(128)
real_df_class_false.describe()

,duration,creditamount,installmentrate,residencesince,age,existingcredits,peopleliable,classification,existingchecking_A11,existingchecking_A12,...,housing_A152,housing_A153,job_A171,job_A172,job_A173,job_A174,telephone_A191,telephone_A192,foreignworker_A201,foreignworker_A202
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.0,128.000000,128.000000,...,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000
mean,20.062500,2831.937500,2.968750,2.890625,36.507812,1.484375,1.140625,0.0,0.203125,0.257812,...,0.796875,0.070312,0.039062,0.140625,0.664062,0.156250,0.585938,0.414062,0.945312,0.054688
std,10.979401,2346.876379,1.107864,1.081224,11.478152,0.651882,0.349000,0.0,0.403906,0.439149,...,0.403906,0.256678,0.194505,0.349000,0.474173,0.364519,0.494495,0.494495,0.228263,0.228263
min,4.000000,250.000000,1.000000,1.000000,20.000000,1.000000,1.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12.000000,1436.250000,2.000000,2.000000,27.750000,1.000000,1.000000,0.0,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,18.000000,2227.500000,3.000000,3.000000,35.000000,1.000000,1.000000,0.0,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,24.000000,3390.500000,4.000000,4.000000,40.250000,2.000000,1.000000,0.0,0.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,60.000000,13756.000000,4.000000,4.000000,74.000000,4.000000,2.000000,0.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
gan_generator = GANDataGenerator(dataset=origin_dataset, model_class=CGAN, trained_model_path=)